## BANKING CUSTOMER SUPPORT APPLICATION

This file creates the Banking Support application
Tech Stack
1. Python
2. LLM OpenAI, gpt-4
3. Can do a Heuristic Analysis if LLM is unavailable.
4. Streamlit as UI

## Initial Setup

### Create Folders

In [1]:
import os
os.makedirs("agents", exist_ok=True)
os.makedirs("db", exist_ok=True)
os.makedirs("tests", exist_ok=True)
os.makedirs("logs", exist_ok=True)
print ("Folders created..")

Folders created..


### Create DB

In [2]:
%%writefile db/schema.sql
-- db/schema.sql

CREATE TABLE IF NOT EXISTS support_tickets (
    ticket_id      TEXT PRIMARY KEY,              -- 6-digit ID (e.g., "123456")
    customer_name  TEXT,                          -- optional, may be NULL
    message        TEXT NOT NULL,                 -- original feedback / complaint
    status         TEXT NOT NULL DEFAULT 'Open',  -- Open / In Progress / Resolved
    correlation_id TEXT,                          -- Transaction ID for Ops
    created_at     TEXT NOT NULL,                 -- ISO datetime string
    updated_at     TEXT NOT NULL                  -- ISO datetime string
);

Overwriting db/schema.sql


## Creating Application

### db/db_utils.py

In [3]:
%%writefile db/db_utils.py
import sqlite3
from pathlib import Path
from datetime import datetime

# Path to the SQLite database file (support.db will be created automatically)
DB_PATH = Path(__file__).resolve().parent / "support.db"


def get_connection():
    """Return a SQLite connection to the support.db file."""
    conn = sqlite3.connect(DB_PATH)
    # So we can access columns by name (row["ticket_id"])
    conn.row_factory = sqlite3.Row
    return conn


def init_db():
    """Create the support_tickets table if it doesn't exist."""
    schema_file = Path(__file__).resolve().parent / "schema.sql"
    with get_connection() as conn, open(schema_file, "r", encoding="utf-8") as f:
        conn.executescript(f.read())
    print("✅ Database initialized (support_tickets table ready).")


def create_ticket(ticket_id: str, message: str, customer_name: str | None = None, correlation_id: str | None = None) -> None:
    now = datetime.utcnow().isoformat()
    with get_connection() as conn:
        conn.execute(
            """
            INSERT INTO support_tickets
            (ticket_id, customer_name, message, status, created_at, updated_at, correlation_id)
            VALUES (?, ?, ?, ?, ?, ?, ?)
            """,
            (ticket_id, customer_name, message, "Open", now, now, correlation_id),
        )
    print(f"✅ Ticket {ticket_id} created.")


def get_ticket(ticket_id: str) -> dict | None:
    """Fetch a ticket by ticket_id. Return dict or None if not found."""
    with get_connection() as conn:
        cur = conn.execute(
            "SELECT ticket_id, customer_name, message, status, created_at, updated_at "
            "FROM support_tickets WHERE ticket_id = ?",
            (ticket_id,),
        )
        row = cur.fetchone()

    if row is None:
        return None

    return dict(row)


def update_ticket_status(ticket_id: str, new_status: str) -> None:
    """Update the status of an existing ticket."""
    now = datetime.utcnow().isoformat()
    with get_connection() as conn:
        conn.execute(
            """
            UPDATE support_tickets
            SET status = ?, updated_at = ?
            WHERE ticket_id = ?
            """,
            (new_status, now, ticket_id),
        )
    print(f"✅ Ticket {ticket_id} status updated to {new_status}.")


Overwriting db/db_utils.py


#### Test DB

In [ ]:
from db.db_utils import init_db, create_ticket, get_ticket, update_ticket_status

# 1. Initialize DB and table
init_db()

import time
ticket_id = str(int(time.time() * 1000) % 1_000_000).zfill(6)

# 2. Create a sample ticket
create_ticket(ticket_id, "ATM did not dispense cash but account debited.", "Test User")

# 3. Fetch and print the ticket
ticket = get_ticket(ticket_id)
print("Fetched ticket:", ticket)

# 4. Update the status
update_ticket_status(ticket_id, "In Progress")

# 5. Fetch again to see updated status
ticket_updated = get_ticket(ticket_id)
print("Updated ticket:", ticket_updated)

In [ ]:
%%writefile agents/__init__.py
# Makes agents a Python package

### agents/logger.py

In [4]:
%%writefile agents/logger.py
import logging
import os

def setup_logging():
    os.makedirs("logs", exist_ok=True)

    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s | %(levelname)s | %(name)s | %(message)s",
        handlers=[
            logging.FileHandler("logs/app.log"),
            logging.StreamHandler()
        ]
    )

Overwriting agents/logger.py


### agents/classifier_agent.py

In [5]:
%%writefile agents/classifier_agent.py
"""
Classifier Agent (LLM-backed with heuristic fallback)

This module exposes classify_message(message, use_llm=True) -> str
Possible return values:
- POSITIVE_FEEDBACK
- NEGATIVE_FEEDBACK
- QUERY

Behavior:
- If use_llm=True, attempts to call the OpenAI Chat API to classify the message.
- If the API call fails, times out, or returns an unexpected answer, falls back
  to the original rule-based heuristic classifier.
- The model name can be set via environment variable OPENAI_MODEL (default 'gpt-4').
- Requires OPENAI_API_KEY in environment for LLM usage.
"""

from __future__ import annotations
import os
import re
import logging
import time
from typing import Optional


logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

# Local heuristic fallback (keeps previous behavior)
def _contains_any(text: str, keywords: list[str]) -> bool:
    text_lower = text.lower()
    return any(k in text_lower for k in keywords)

def heuristic_classify(message: str) -> str:
    msg = message.strip()
    msg_lower = msg.lower()

    # 1) Question cues → QUERY
    question_words = ["how", "when", "what", "where", "why", "status", "ticket", "help", "can you", "could you"]
    if "?" in msg or _contains_any(msg, question_words):
        return "QUERY"

    # 2) Negative sentiment cues
    negative_words = [
        "not happy", "unhappy", "angry", "bad", "worst", "terrible", "horrible",
        "complain", "complaint", "issue", "problem", "frustrated", "disappointed",
        "did not work", "didn't work", "didnt work", "money deducted", "debited but"
    ]
    if _contains_any(msg, negative_words):
        return "NEGATIVE_FEEDBACK"

    # 3) Positive sentiment cues
    positive_words = [
        "thank you", "thanks", "great", "awesome", "good service", "well done",
        "happy", "satisfied", "love the service"
    ]
    if _contains_any(msg, positive_words):
        return "POSITIVE_FEEDBACK"

    # 4) Fallback: treat as query
    return "QUERY"


# ---------- LLM integration ----------
# Attempt to import openai only when required to avoid import errors if not installed.
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-4")  # gpt-3.5-turbo; change if needed
OPENAI_TIMEOUT = float(os.getenv("OPENAI_TIMEOUT", "10"))  # seconds



def _safe_import_openai():
    try:
        import openai  # type: ignore
        return openai
    except Exception as e:
        logger.warning("openai package not available in the environment (%s).", e)
        return None


def _call_openai_chat(message: str, model: str = OPENAI_MODEL, timeout: float = OPENAI_TIMEOUT) -> Optional[str]:
    """
    Calls OpenAI ChatCompletion with a constrained prompt asking for a single label.
    Returns the raw text response or None on failure.
    """
    openai = _safe_import_openai()
    if openai is None:
        return None

    if not OPENAI_API_KEY:
        logger.warning("OPENAI_API_KEY not found in environment; skipping LLM call.")
        return None

    # Configure api key from env (openai picks it up automatically, but set explicitly for clarity)
    openai.api_key = OPENAI_API_KEY

    system_prompt = (
        "You are a strict classifier. Classify the user's message into EXACTLY ONE of the following labels:\n"
        "- POSITIVE_FEEDBACK\n"
        "- NEGATIVE_FEEDBACK\n"
        "- QUERY\n\n"
        "Return only the label, and nothing else (no punctuation, no explanation). "
        "If the message contains both a complaint and a question, prefer QUERY only if the user explicitly asks about ticket status; "
        "otherwise prefer NEGATIVE_FEEDBACK for complaints. Be concise and deterministic."
    )

    user_prompt = (
        f"Message: \"{message}\"\n\n"
        "Which single label from the list applies? Reply with only the label."
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    # We'll do a small retry loop for transient errors
    retries = 2
    for attempt in range(1, retries + 1):
        try:
            # Use ChatCompletion API
            resp = openai.ChatCompletion.create(
                model=model,
                messages=messages,
                temperature=0.0,
                max_tokens=8,
                n=1,
                request_timeout=timeout,
            )
            # Extract text
            content = ""
            if resp and "choices" in resp and len(resp["choices"]) > 0:
                content = resp["choices"][0]["message"]["content"].strip()
            if content:
                return content
            else:
                logger.warning("Empty response from OpenAI on attempt %s", attempt)
        except Exception as e:
            logger.warning("OpenAI call failed on attempt %s: %s", attempt, e)
            # small backoff
            time.sleep(0.5 * attempt)

    return None


def _normalize_label(text: str) -> Optional[str]:
    """
    Normalize the model output to one of the three canonical labels.
    If it cannot be mapped, return None.
    """
    if not text:
        return None

    t = text.strip().upper()
    # direct matches
    for lbl in ("POSITIVE_FEEDBACK", "NEGATIVE_FEEDBACK", "QUERY"):
        if t == lbl:
            return lbl

    # Accept some common shorthand or accidental punctuation
    t_clean = re.sub(r"[^A-Z_]", "", t)  # remove punctuation / numbers, keep letters & underscore
    if t_clean in ("POSITIVEFEEDBACK", "POSITIVE_FEEDBACK", "POSITIVE", "THANKS"):
        return "POSITIVE_FEEDBACK"
    if t_clean in ("NEGATIVEFEEDBACK", "NEGATIVE_FEEDBACK", "NEGATIVE", "COMPLAINT", "COMPLAIN"):
        return "NEGATIVE_FEEDBACK"
    if t_clean in ("QUERY", "QUESTION", "STATUS", "TICKET"):
        return "QUERY"

    # try to detect keywords if LLM gave a short phrase
    t_lower = t.lower()
    if any(k in t_lower for k in ["thank", "thanks", "great", "good", "happy", "satisfied"]):
        return "POSITIVE_FEEDBACK"
    if any(k in t_lower for k in ["not", "unhappy", "angry", "complain", "debit", "issue", "problem", "frustrat"]):
        return "NEGATIVE_FEEDBACK"
    if any(k in t_lower for k in ["?", "how", "what", "when", "status", "ticket", "help"]):
        return "QUERY"

    return None


def classify_message(message: str, use_llm: bool = True) -> str:
    """
    Public function to classify a message.

    Parameters:
    - message: the customer message
    - use_llm: if True, try LLM first; if False, use heuristic only.

    Returns one of:
    - 'POSITIVE_FEEDBACK', 'NEGATIVE_FEEDBACK', 'QUERY'
    """
    message = (message or "").strip()
    if not message:
        logger.info("Empty message provided to classify_message; returning QUERY by default.")
        return "QUERY"

    # If LLM usage is disabled explicitly, use heuristic directly
    if not use_llm:
        return heuristic_classify(message)

    # Try LLM
    raw = _call_openai_chat(message)
    if raw:
        label = _normalize_label(raw)
        if label:
            logger.info("LLM classification succeeded: %s -> %s", message[:60], label)
            return label
        else:
            logger.warning("LLM returned unexpected label '%s'; falling back to heuristic.", raw)

    # Fallback to heuristic
    fallback = heuristic_classify(message)
    logger.info("Heuristic fallback classification: %s -> %s", message[:60], fallback)
    return fallback


# Keep the old label constants for compatibility with existing code
POSITIVE_FEEDBACK = "POSITIVE_FEEDBACK"
NEGATIVE_FEEDBACK = "NEGATIVE_FEEDBACK"
QUERY = "QUERY"


# Quick CLI-like test when run as script
if __name__ == "__main__":
    import sys
    msg = " ".join(sys.argv[1:]) if len(sys.argv) > 1 else "I want to know the status of my ticket 123456"
    print("Input:", msg)
    print("Classification (LLM preferred):", classify_message(msg, use_llm=True))
    print("Classification (heuristic only):", classify_message(msg, use_llm=False))

Overwriting agents/classifier_agent.py


### agents/classifier_agent_wo_llm.py

In [6]:
%%writefile agents/classifier_agent_wo_llm.py
"""
Classifier Agent

Classifies incoming customer messages into one of:
- "POSITIVE_FEEDBACK"
- "NEGATIVE_FEEDBACK"
- "QUERY"

Currently uses simple rule-based heuristics.
Later you can plug in an LLM inside `classify_message()`.
"""

import re


POSITIVE_FEEDBACK = "POSITIVE_FEEDBACK"
NEGATIVE_FEEDBACK = "NEGATIVE_FEEDBACK"
QUERY = "QUERY"


def _contains_any(text: str, keywords: list[str]) -> bool:
    text_lower = text.lower()
    return any(k in text_lower for k in keywords)


def classify_message(message: str) -> str:
    """
    Classify a customer message into one of the three classes.

    Heuristic baseline:
    - If it clearly looks like a question (question mark, wh-words) → QUERY
    - Else if strong negative words → NEGATIVE_FEEDBACK
    - Else if positive words → POSITIVE_FEEDBACK
    - Else fallback: QUERY
    """
    msg = message.strip()
    msg_lower = msg.lower()

    # 1) Question cues → QUERY
    question_words = ["how", "when", "what", "where", "why", "status", "ticket", "help", "can you", "could you"]
    if "?" in msg or _contains_any(msg, question_words):
        return QUERY

    # 2) Negative sentiment cues
    negative_words = [
        "not happy", "unhappy", "angry", "bad", "worst", "terrible", "horrible",
        "complain", "complaint", "issue", "problem", "frustrated", "disappointed",
        "did not work", "didn't work", "didnt work", "money deducted", "debited but"
    ]
    if _contains_any(msg, negative_words):
        return NEGATIVE_FEEDBACK

    # 3) Positive sentiment cues
    positive_words = [
        "thank you", "thanks", "great", "awesome", "good service", "well done",
        "happy", "satisfied", "love the service"
    ]
    if _contains_any(msg, positive_words):
        return POSITIVE_FEEDBACK

    # 4) Fallback: treat as query
    return QUERY

Overwriting agents/classifier_agent_wo_llm.py


### agents/feedback_agent.py

In [7]:
%%writefile agents/feedback_agent.py
"""
Feedback Agent

Handles:
- POSITIVE_FEEDBACK: send a warm thank-you message.
- NEGATIVE_FEEDBACK: create a ticket in the DB, return an empathetic reply.
"""

import random
import logging
from typing import Optional

from db.db_utils import create_ticket, get_ticket
from agents.classifier_agent import POSITIVE_FEEDBACK, NEGATIVE_FEEDBACK

logger = logging.getLogger(__name__)

def _generate_unique_ticket_id() -> str:
    """Generate a unique 6-digit ticket ID that is not already in the DB."""
    from db.db_utils import get_ticket  # local import to avoid cycles

    while True:
        ticket_id = f"{random.randint(0, 999999):06d}"
        existing = get_ticket(ticket_id)
        if existing is None:
            return ticket_id


def handle_feedback(message: str, sentiment: str, customer_name: Optional[str] = None, correlation_id=None) -> dict:
    """
    Handle feedback based on sentiment.

    Returns a dict with:
    - "reply": str
    - "ticket_id": str | None
    - "sentiment": str
    """
    logger.info(
        "FEEDBACK_AGENT_STARTED | correlation_id=%s | sentiment=%s",
        correlation_id,
        sentiment
    )
    customer_display = customer_name or "Valued Customer"

    if sentiment == POSITIVE_FEEDBACK:
        reply = (
            f"Thank you for your kind feedback, {customer_display}! "
            "We really appreciate you taking the time to share this with us."
        )
        return {
            "reply": reply,
            "ticket_id": None,
            "sentiment": sentiment,
        }

    if sentiment == NEGATIVE_FEEDBACK:
        ticket_id = _generate_unique_ticket_id()
        # Store the ticket
        create_ticket(
            ticket_id=ticket_id,
            message=message,
            customer_name=customer_name,
            correlation_id=correlation_id,
        )
        reply = (
            f"We're really sorry to hear about your experience, {customer_display}. "
            f"I've created a support ticket for you: #{ticket_id}. "
            "Our team will review this and get back to you as soon as possible."
        )
        return {
            "reply": reply,
            "ticket_id": ticket_id,
            "sentiment": sentiment,
        }

    # Fallback (should not normally happen)
    return {
        "reply": "Thank you for your message.",
        "ticket_id": None,
        "sentiment": sentiment,
    }

Overwriting agents/feedback_agent.py


### agents/query_agent.py

In [8]:
%%writefile agents/query_agent.py
"""
Query Agent

Handles queries about ticket status.
Extracts a 6-digit ticket number from the message and looks it up in the DB.
"""

import re
import logging
from db.db_utils import get_ticket

logger = logging.getLogger(__name__)

TICKET_PATTERN = re.compile(r"\b(\d{6})\b")


def _extract_ticket_id(message: str) -> str | None:
    """Return the first 6-digit sequence found in the message, or None."""
    match = TICKET_PATTERN.search(message)
    if match:
        return match.group(1)
    return None


def handle_query(message: str, correlation_id=None) -> dict:
    """
    Handle a query about ticket status.

    Returns dict:
    - "reply": str
    - "ticket_id": str | None
    - "found": bool
    """
    logger.info(
        "QUERY_AGENT_STARTED | correlation_id=%s",
        correlation_id,
    )
    ticket_id = _extract_ticket_id(message)

    if not ticket_id:
        return {
            "reply": (
                "I couldn't find a ticket number in your message. "
                "Please share your 6-digit ticket ID so I can check the status for you."
            ),
            "ticket_id": None,
            "found": False,
        }

    ticket = get_ticket(ticket_id)

    logger.info(
        "TICKET_LOOKUP_COMPLETED | correlation_id=%s | ticket_id=%s | found=%s",
        correlation_id,
        ticket_id,
        bool(ticket),
    )

    if ticket is None:
        return {
            "reply": f"I couldn't find any ticket with ID #{ticket_id}. "
                     "Please verify the number and try again.",
            "ticket_id": ticket_id,
            "found": False,
        }

    status = ticket["status"]
    return {
        "reply": f"Your ticket #{ticket_id} is currently marked as: {status}.",
        "ticket_id": ticket_id,
        "found": True,
    }

Overwriting agents/query_agent.py


### orchestrator.py

In [9]:
%%writefile orchestrator.py
"""
Orchestrator

Wires together:
- Classifier Agent
- Feedback Agent
- Query Agent
"""

from typing import Optional

from agents.classifier_agent import (
    classify_message,
    POSITIVE_FEEDBACK,
    NEGATIVE_FEEDBACK,
    QUERY,
)
from agents.feedback_agent import handle_feedback
from agents.query_agent import handle_query


def process_customer_message(message: str, customer_name: Optional[str] = None) -> dict:
    """
    Main entry point for processing a customer message.

    Returns a dict with:
    - "original_message": str
    - "classification": str  (one of the 3 labels)
    - "handled_by": str      ("FeedbackAgent" or "QueryAgent")
    - "reply": str
    - "ticket_id": str | None
    - Additional info per agent if needed
    """
    classification = classify_message(message)

    if classification in {POSITIVE_FEEDBACK, NEGATIVE_FEEDBACK}:
        result = handle_feedback(message=message, sentiment=classification, customer_name=customer_name)
        return {
            "original_message": message,
            "classification": classification,
            "handled_by": "FeedbackAgent",
            "reply": result["reply"],
            "ticket_id": result["ticket_id"],
        }

    if classification == QUERY:
        result = handle_query(message=message)
        return {
            "original_message": message,
            "classification": classification,
            "handled_by": "QueryAgent",
            "reply": result["reply"],
            "ticket_id": result["ticket_id"],
        }

    # Fallback (shouldn't happen)
    return {
        "original_message": message,
        "classification": classification,
        "handled_by": "Unknown",
        "reply": "I'm not sure how to handle this message.",
        "ticket_id": None,
    }


if __name__ == "__main__":
    # Simple manual test
    samples = [
        "Thank you, the app is really good!",
        "I am very disappointed, my money was debited but I didn't receive cash.",
        "What is the status of my ticket 123456?",
    ]
    for msg in samples:
        print("\nMessage:", msg)
        out = process_customer_message(msg, customer_name="Test User")
        print("Result:", out)

Overwriting orchestrator.py


### app_streamlit.py

In [10]:
%%writefile app_streamlit.py
# Setup logging
from agents.logger import setup_logging
setup_logging()

import streamlit as st
import pandas as pd
import logging

from db.db_utils import init_db, get_connection
from orchestrator import process_customer_message
from langgraph_impl import run_support_graph


# Initialize DB on app start
init_db()


def load_recent_tickets(limit: int = 20) -> pd.DataFrame:
    """Fetch recent tickets from the DB as a DataFrame."""
    conn = get_connection()
    try:
        query = """
        SELECT ticket_id, customer_name, message, status, created_at, updated_at
        FROM support_tickets
        ORDER BY created_at DESC
        LIMIT ?
        """
        df = pd.read_sql_query(query, conn, params=(limit,))
    finally:
        conn.close()
    return df


def main():
    st.set_page_config(page_title="Banking Support AI", page_icon="💬")

    st.title("💬 Banking Customer Support AI (Multi-Agent)")
    st.write(
        "This app classifies customer messages as **positive feedback**, "
        "**negative feedback**, or **query**, and routes them to the "
        "appropriate agent."
    )

    st.sidebar.header("About")
    st.sidebar.markdown(
        """
        **Agents used:**
        - Classifier Agent
        - Feedback Agent (positive / negative)
        - Query Agent (ticket status)
        """
    )

    st.header("1️⃣ Send a Customer Message")

    customer_name = st.text_input("Customer name (optional)", value="")
    message = st.text_area("Customer message", height=150)

    if st.button("Process Message"):
        if not message.strip():
            st.warning("Please enter a message before processing.")
        else:
            # when user clicks "Process Message"
            final_state = run_support_graph(message=message, customer_name=customer_name or None)
            reply = final_state.get("reply", "No response generated. Please retry after sometime")
            
            # Use final_state fields to show UI
            # st.write("Classification:", final_state.get("classification"))
            # st.write("Handled by:", final_state.get("handled_by"))
            # st.info(reply)
            if final_state.get("ticket_id"):
                st.success(f"Ticket ID: #{final_state.get('ticket_id')}")
            # --------------------
            # Simple Orchestrator call

#            result = process_customer_message(
#                message=message,
#                customer_name=customer_name or None,
#            )
#
#            st.subheader("Result")
#            st.write(f"**Classification:** `{result['classification']}`")
#            st.write(f"**Handled by:** `{result['handled_by']}`")
#
#            if result.get("ticket_id"):
#                st.success(f"Ticket ID: #{result['ticket_id']}")

             # -------------------------------

            st.markdown("**Reply to customer:**")
            st.info(reply)

    st.header("2️⃣ Recent Support Tickets")
    with st.expander("Show recent tickets"):
        try:
            df = load_recent_tickets(limit=20)
            if df.empty:
                st.write("No tickets found yet.")
            else:
                st.dataframe(df)
        except Exception as e:
            st.error(f"Error loading tickets: {e}")


if __name__ == "__main__":
    main()

Overwriting app_streamlit.py


### langgraph_impl.py

In [11]:
%%writefile langgraph_impl.py
# langgraph_impl.py
from typing_extensions import TypedDict
from datetime import datetime
import logging

from langgraph.graph import START, END, StateGraph
from langgraph.checkpoint.memory import MemorySaver
import uuid

from agents.classifier_agent import (
    classify_message,
    POSITIVE_FEEDBACK,
    NEGATIVE_FEEDBACK,
    QUERY,
)
from agents.feedback_agent import handle_feedback
from agents.query_agent import handle_query


logger = logging.getLogger(__name__)

# -------------------------------------------------
# 1) Define Graph State
# -------------------------------------------------
class SupportState(TypedDict, total=False):
    message: str
    customer_name: str | None
    correlation_id: str
    classification: str
    handled_by: str
    ticket_id: str | None
    reply: str
    processed_at: str


# -------------------------------------------------
# 2) Define Nodes
# -------------------------------------------------
def node_classifier(state: SupportState) -> dict:
    logger.info(
        "CLASSIFICATION_STARTED | correlation_id=%s | message=%s",
        state["correlation_id"],
        state["message"][:80],
    )
    cls = classify_message(state["message"])
    logger.info(
        "CLASSIFICATION_COMPLETED | correlation_id=%s | classification=%s",
        state["correlation_id"],
        cls,
    )
    return {"classification": cls}


def node_feedback(state: SupportState) -> dict:
    logger.info(
        "Running Feedback Agent | correlation_id=%s",
        state["correlation_id"]
    )
    result = handle_feedback(
        message=state["message"],
        sentiment=state["classification"],
        customer_name=state.get("customer_name"),
        correlation_id=state["correlation_id"],
    )
    return {
        "reply": result["reply"],
        "ticket_id": result["ticket_id"],
        "handled_by": "FeedbackAgent",
        "processed_at": datetime.utcnow().isoformat(),
    }


def node_query(state: SupportState) -> dict:
    logger.info("Running Query Agent")
    result = handle_query(
      message=state["message"],
      correlation_id=state["correlation_id"],
    )
    return {
        "reply": result["reply"],
        "ticket_id": result["ticket_id"],
        "handled_by": "QueryAgent",
        "processed_at": datetime.utcnow().isoformat(),
    }


# -------------------------------------------------
# 3) Routing Logic
# -------------------------------------------------
def route_after_classification(state: SupportState) -> str:
    cls = state["classification"]
    logger.info(
        "ROUTING_DECISION | correlation_id=%s | next_agent=%s",
        state["correlation_id"],
        "FeedbackAgent" if cls != QUERY else "QueryAgent",
    )

    if cls in (POSITIVE_FEEDBACK, NEGATIVE_FEEDBACK):
        return "feedback"
    return "query"


# -------------------------------------------------
# 4) Build Graph WITH Checkpointing
# -------------------------------------------------
def build_support_graph():
    graph = StateGraph(SupportState)

    graph.add_node("classifier", node_classifier)
    graph.add_node("feedback", node_feedback)
    graph.add_node("query", node_query)

    graph.add_edge(START, "classifier")

    graph.add_conditional_edges(
        "classifier",
        route_after_classification,
        {
            "feedback": "feedback",
            "query": "query",
        },
    )

    graph.add_edge("feedback", END)
    graph.add_edge("query", END)

    # CHECKPOINTING ADDITION
    checkpointer = MemorySaver()

    compiled_graph = graph.compile(checkpointer=checkpointer)

    return compiled_graph, checkpointer


# Build once
GRAPH, CHECKPOINTER = build_support_graph()


# -------------------------------------------------
# 5) Public Runner
# -------------------------------------------------
def run_support_graph(message: str, customer_name: str | None = None) -> dict:
    correlation_id = f"cs-{uuid.uuid4()}"
    """
    Runs the LangGraph with checkpointing enabled.
    """
    logger.info(
        "REQUEST_RECEIVED | correlation_id=%s | customer=%s",
        correlation_id,
        customer_name or "NA",
    )
    initial_state: SupportState = {
        "message": message,
        "customer_name": customer_name,
        "correlation_id": correlation_id,
    }

    # Each run MUST have a thread_id for checkpointing
    config = {
        "configurable": {
            "thread_id": correlation_id
        }
    }

    final_state = GRAPH.invoke(initial_state, config=config)
    logger.info(
      "REQUEST_COMPLETED | correlation_id=%s | handled_by=%s",
      correlation_id,
      final_state.get("handled_by"),
    )
    return dict(final_state)


Overwriting langgraph_impl.py


# Testing

In [ ]:
from langgraph_impl import run_support_graph
from db.db_utils import init_db, get_ticket

init_db()
res = run_support_graph("I am very unhappy, money was debited but ATM did not give cash.", "Rajesh")
print(res)
# check ticket created if negative
print(get_ticket(res.get("ticket_id")))


 ### Test Heuristic Classifier 
 ##### ONLY IF LLM IS UNAVAILABLE 

In [ ]:
from db.db_utils import init_db, get_ticket
from orchestrator import process_customer_message

# 1. Ensure DB/table exists
init_db()

# 2. Positive feedback test
out_pos = process_customer_message("Thank you, the service was awesome!", customer_name="Rajesh")
print("POSITIVE TEST:\n", out_pos)

# 3. Negative feedback -> ticket should be created
out_neg = process_customer_message(
    "I am very unhappy, money was debited but the ATM did not give cash.",
    customer_name="Rajesh",
)
print("\nNEGATIVE TEST:\n", out_neg)

neg_ticket_id = out_neg["ticket_id"]
print("Created ticket ID:", neg_ticket_id)
print("Ticket from DB:", get_ticket(neg_ticket_id))

# 4. Query test with that ticket
query_msg = f"Hi, what is the status of my ticket {neg_ticket_id}?"
out_query = process_customer_message(query_msg, customer_name="Rajesh")
print("\nQUERY TEST:\n", out_query)

## Get DB Entries

In [ ]:
import sqlite3
from pathlib import Path

DB_PATH = Path("db/support.db")

conn = sqlite3.connect(DB_PATH)
conn.row_factory = sqlite3.Row

cursor = conn.execute("SELECT * FROM support_tickets")
rows = cursor.fetchall()

for r in rows:
    print(dict(r))

In [ ]:
from agents.classifier_agent import classify_message

# quick tests
print(classify_message("Thank you! Your service is great.", use_llm=True))
print(classify_message("I am furious: my account was debited but I got no cash.", use_llm=True))
print(classify_message("The amount debitted is correct. share the creditor", use_llm=False))
print(classify_message("What is the status of my ticket 123456?", use_llm=True))

# If your API key isn't set or you want to test the fallback:
print(classify_message("What is the status of my ticket 123456?", use_llm=False))

# Generate Test Harness

In [ ]:
%%writefile tests/test_harness.py
# tests/test_harness.py

import sys
import os

# Ensure project root is on Python path
PROJECT_ROOT = os.path.abspath(os.path.join(os.path.dirname(__file__), ".."))
if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
"""
System Test Harness for Banking Support AI (LangGraph-based)

Runs a predefined set of customer messages through the system
and prints:
- classification
- agent routing
- ticket creation
- DB validation
- correlation_id (for ops tracing)

This demonstrates end-to-end correctness and LLMOps observability.
"""

from langgraph_impl import run_support_graph
from db.db_utils import get_ticket, init_db
from agents.logger import setup_logging
setup_logging()   # safe for test execution

# -------------------------------------------------
# 1) Test cases (realistic banking scenarios)
# -------------------------------------------------
TEST_CASES = [
    {
        "id": 1,
        "message": "Thank you, your mobile banking app is excellent!",
        "type": "POSITIVE_FEEDBACK",
    },
    {
        "id": 2,
        "message": "Money was debited from my account but ATM did not dispense cash.",
        "type": "NEGATIVE_FEEDBACK",
    },
    {
        "id": 3,
        "message": "What is the status of my ticket 123456?",
        "type": "QUERY",
    },
    {
        "id": 4,
        "message": "Very unhappy with the service. This is unacceptable.",
        "type": "NEGATIVE_FEEDBACK",
    },
    {
        "id": 5,
        "message": "How can I reset my debit card PIN?",
        "type": "QUERY",
    },
    {
        "id": 6,
        "message": "Great support from your call center team.",
        "type": "POSITIVE_FEEDBACK",
    },
    {
        "id": 7,
        "message": "I have an issue with an incorrect charge on my account.",
        "type": "NEGATIVE_FEEDBACK",
    },
    {
        "id": 8,
        "message": "Can you help me check the status of my complaint?",
        "type": "QUERY",
    },
    {
        "id": 9,
        "message": "I am frustrated. My account was debited twice.",
        "type": "NEGATIVE_FEEDBACK",
    },
    {
        "id": 10,
        "message": "Thanks for resolving my issue quickly!",
        "type": "POSITIVE_FEEDBACK",
    },
]

# -------------------------------------------------
# 2) Run harness
# -------------------------------------------------
def run_test_harness():
    print("=" * 80)
    print("BANKING SUPPORT AI – SYSTEM TEST HARNESS")
    print("=" * 80)

    init_db()

    for test in TEST_CASES:
        print("\n" + "-" * 80)
        print(f"TEST CASE {test['id']}")
        print(f"Input Message : {test['message']}")

        result = run_support_graph(
            message=test["message"],
            customer_name="TestUser",
        )

        print(f"Classification : {result.get('classification')}")
        print(f"Handled By     : {result.get('handled_by')}")
        print(f"Reply          : {result.get('reply')}")
        print(f"Correlation ID : {result.get('correlation_id', 'N/A')}")

        ticket_id = result.get("ticket_id")

        if ticket_id:
            print(f"Ticket Created : {ticket_id}")
            ticket = get_ticket(ticket_id)
            if ticket:
                print("DB Verification: SUCCESS")
                print(
                    f"  DB.ticket_id={ticket['ticket_id']} | "
                    f"status={ticket['status']} | "
                    f"correlation_id={ticket['correlation_id']}"
                )
            else:
                print("DB Verification: FAILED (ticket not found)")
        else:
            print("Ticket Created : NO")

    print("\n" + "=" * 80)
    print("TEST HARNESS COMPLETED")
    print("=" * 80)


if __name__ == "__main__":
    run_test_harness()


In [ ]:
!python tests/test_harness.py

## Test Checkpointing

In [ ]:
from langgraph_impl import run_support_graph, CHECKPOINTER

res = run_support_graph(
    "Money debited but ATM did not dispense cash",
    customer_name="Rajesh",
)

print("Final Output:")
print(res)

In [ ]:
from datetime import datetime
thread_id = f"run-{datetime.utcnow().isoformat()}"
print (thread_id)

In [ ]:
from langgraph_impl import GRAPH

initial_state = {
    "message": "Money debited but ATM did not dispense cash",
    "customer_name": "Rajesh"
}

config = {
    "configurable": {
        "thread_id": thread_id
    }
}

final_state = GRAPH.invoke(initial_state, config=config)
print("Final state:", final_state)


In [ ]:
from langgraph_impl import CHECKPOINTER

print("\nCheckpointed states for thread:", thread_id)

for checkpoint in CHECKPOINTER.list(config):
    print(checkpoint)


## Langraph reload

In [ ]:
import langgraph_impl
dir(langgraph_impl)

In [ ]:
print("\nCheckpointed states:")
for checkpoint in CHECKPOINTER.list():
    print(checkpoint)

In [ ]:
import importlib
import langgraph_impl

importlib.reload(langgraph_impl)

dir(langgraph_impl)


# Launch Streamlit

In [ ]:
!streamlit run app_streamlit.py




  You can now view your Streamlit app in your browser.

  Network URL: http://172.16.0.3:8501
  External URL: http://34.208.97.119:8501


  A new version of Streamlit is available.

  See what's new at https://discuss.streamlit.io/c/announcements

  Enter the following command to upgrade:
  $ pip install streamlit --upgrade



## Misclaneous

In [ ]:
!pkill -f streamlit

In [ ]:
!ps aux | grep streamlit

In [ ]:
!pip install -U langgraph